# TODO

- ✅ cosa scegliere tra [**pm4py**](https://processintelligence.solutions/static/api/2.7.11/pm4py.html#pm4py.convert.convert_log_to_ocel) e ocpa
- ✅ importa json da drive
- ✅ estrarre dal df i vari attributi
  ❌


# Importazione librerie e selezione file


## Se in esecuzione in locale


In [1]:
%pip install pyecore >> /dev/null
%pip install --upgrade pm4py >> /dev/null
%pip install pm4py >> /dev/null
#!pip install web3
import pandas as pd 
import os
import datetime
import json
import pm4py
import shutil
from collections import Counter
import numpy

Note: you may need to restart the kernel to use updated packages.


Impossibile trovare il percorso specificato.


Note: you may need to restart the kernel to use updated packages.


Impossibile trovare il percorso specificato.


Note: you may need to restart the kernel to use updated packages.


Impossibile trovare il percorso specificato.


## Flow app

In [2]:
folder_path = "Datasets"

file_list = [file for file in os.listdir(
    folder_path) if file.endswith(".json")]
print("File JSON disponibili:")
for i, file_name in enumerate(file_list):
    print(f"{i}: {file_name}")

if file_list:
    selected_index = int(
        input("Inserisci il numero corrispondente al file che vuoi scegliere: "))
    selected_file = file_list[selected_index]

    print(f"Hai selezionato: {selected_file}")
    file_path = os.path.join(folder_path, selected_file)
    print(f"Percorso completo del file: {file_path}")
else:
    print("Non ci sono file .json nella cartella.")

File JSON disponibili:
0: pancacke100txs.json
Hai selezionato: pancacke100txs.json
Percorso completo del file: Datasets\pancacke100txs.json


In [40]:

with open(file_path, "r") as file:
    data = json.load(file)

meta_fields = ["txHash", "blockNumber", "contractAddress",
               "sender", "gasUsed", "activity", "timestamp"]


def normalize_data(data):
    # Normalizza i dati senza aggregazione
    inputs_df = pd.json_normalize(
        data, record_path="inputs", meta=meta_fields, record_prefix="input_")
    inputs_df["input__id"] = "input_" + inputs_df["txHash"] + "_" + \
        (inputs_df.groupby("txHash").cumcount() + 1).astype(str)

    events_df = pd.json_normalize(
        data, record_path="events", meta=meta_fields, record_prefix="event_")
    events_df["event__id"] = "event_" + events_df["txHash"] + "_" + \
        (events_df.groupby("txHash").cumcount() + 1).astype(str)

    storage_state_df = pd.json_normalize(
        data, record_path="storageState", meta=meta_fields, record_prefix="storageState_")
    storage_state_df["storageState__id"] = "storageState_" + storage_state_df["txHash"] + \
        "_" + (storage_state_df.groupby("txHash").cumcount() + 1).astype(str)

    internal_txs_df = pd.json_normalize(
        data, record_path="internalTxs", meta=meta_fields, record_prefix="internalTx_")
    internal_txs_df["internalTx__id"] = "internalTx_" + internal_txs_df["txHash"] + \
        "_" + (internal_txs_df.groupby("txHash").cumcount() + 1).astype(str)

    # Combina i DataFrame senza aggregazione
    combined_df = pd.concat(
        [inputs_df, events_df, storage_state_df, internal_txs_df], ignore_index=True)

    column_order = [
        "txHash", "blockNumber", "contractAddress", "sender", "gasUsed", "activity", "timestamp",

        "input_inputId", "input_inputName", "input_type", "input_inputValue", "input__id",

        "storageState_variableId", "storageState_variableName", "storageState_type",
        "storageState_variableValue", "storageState_variableRawValue", "storageState__id",

        "internalTx_callId", "internalTx_callType", "internalTx_to",
        "internalTx_inputsCall", "internalTx__id",

        "event_eventId", "event_eventName", "event_eventValues.0", "event_eventValues.1",
        "event_eventValues.2", "event_eventValues.__length__", "event_eventValues.owner",
        "event_eventValues.spender", "event_eventValues.value", "event_eventValues.from",
        "event_eventValues.to", "event_eventValues.3", "event_eventValues._dstChainId",
        "event_eventValues._from", "event_eventValues._toAddress", "event_eventValues._amount",
        "event__id"
    ]

    existing_columns = [
        col for col in column_order if col in combined_df.columns]
    extra_columns = list(set(column_order) - set(existing_columns))

    if extra_columns:
        print(
            f"Warning: The following columns are not present in the data: {extra_columns}")

    return combined_df[existing_columns]


def remove_columns_containing_string(df, substring):
    filtered_columns = [col for col in df.columns if substring not in col]
    return df[filtered_columns]


df = pd.DataFrame(data)

# Esempio di utilizzo della funzione
df_normalized = normalize_data(data)
df_normalized = remove_columns_containing_string(df_normalized, "Id")
df_normalized["timestamp"] = pd.to_datetime(df_normalized["timestamp"])
df_normalized["blockNumber"] = df_normalized["blockNumber"].astype(int)
df_normalized["gasUsed"] = df_normalized["gasUsed"].astype(int)
df_normalized["activity"] = df_normalized["activity"].astype("category")
df_normalized["input_inputName"] = df_normalized["input_inputName"].astype(
    "category")
df_normalized["input_type"] = df_normalized["input_type"].astype("category")
df_normalized["storageState_variableName"] = df_normalized["storageState_variableName"].astype(
    "category")
df_normalized["storageState_type"] = df_normalized["storageState_type"].astype(
    "category")
df_normalized["internalTx_callType"] = df_normalized["internalTx_callType"].astype(
    "category")
df_normalized["event_eventName"] = df_normalized["event_eventName"].astype(
    "category")


def save_file(ocel: pm4py.OCEL, file_name: str):
    year_day = datetime.datetime.utcnow().strftime("%Y_%j")
    hour_minute = datetime.datetime.utcnow().strftime("%H_%M")
    hour_minute = 0  # remove
    complete_file_name = f"{file_name}_{year_day}_{hour_minute}"
    jsonocel_file_path = os.path.join(
        folder_path, f"{os.path.splitext(selected_file)[0]}_{complete_file_name}")
    pm4py.write.write_ocel2_json(ocel, jsonocel_file_path + ".jsonocel")
    shutil.copy(jsonocel_file_path + ".jsonocel", jsonocel_file_path + ".json")

    subfolder_path = os.path.join(
        folder_path, os.path.splitext(selected_file)[0], year_day)
    os.makedirs(subfolder_path, exist_ok=True)
    if os.path.exists(jsonocel_file_path + ".jsonocel"):
        shutil.move(jsonocel_file_path + ".jsonocel", os.path.join(subfolder_path,
                    os.path.basename(jsonocel_file_path + ".jsonocel")))
    else:
        print(
            f"File {jsonocel_file_path + '.jsonocel'} non trovato per lo spostamento.")
    if os.path.exists(jsonocel_file_path + ".json"):
        shutil.move(jsonocel_file_path + ".json", os.path.join(subfolder_path,
                    os.path.basename(jsonocel_file_path + ".json")))
    else:
        print(
            f"File {jsonocel_file_path + '.json'} non trovato per lo spostamento.")
    print(f"File spostati in: {subfolder_path}")


p_log = df_normalized

p_activity = "activity"

p_timestamp = "timestamp"

p_object_types = [
    "contractAddress", "sender", "input_inputName", "storageState_variableName"
]

p_additional_event_attributes = [
    "txHash", "blockNumber", "gasUsed", "input_inputName", "input_type", "input_inputValue",
    "event_eventName", "event_eventValues.0",
    "event_eventValues.1", "event_eventValues.2",
    "event_eventValues.__length__", "event_eventValues.owner",
    "event_eventValues.spender", "event_eventValues.value",
    "event_eventValues.from", "event_eventValues.to", "event_eventValues.3",
    "event_eventValues._from", "event_eventValues._toAddress",
    "event_eventValues._amount", "storageState_variableName",
    "storageState_type", "storageState_variableValue",
    "storageState_variableRawValue", "internalTx_callType", "internalTx_to",
    "internalTx_inputsCall"
]

p_additional_object_attributes = {
    # "txHash": ["txHash", "blockNumber", "contractAddress", "sender", "gasUsed", "activity", "timestamp"],
    # "blockNumber": ["blockNumber"],
    "contractAddress": ["contractAddress"],
    # "sender": ["sender"],
    # "activity": ["activity"],
    "input_inputName": ["input__id", "input_inputName", "input_type", "input_inputValue"],
    "storageState_variableName": ["storageState__id", "storageState_variableName", "storageState_type", "storageState_variableValue", "storageState_variableRawValue"],
    # "internalTx__id": ["internalTx__id", "internalTx_callType", "internalTx_to", "internalTx_inputsCall"],
    # "event__id": ["event__id"]
    # "activity": ["activity"]
}

ocel = pm4py.convert.convert_log_to_ocel(
    log=p_log,
    activity_column=p_activity,
    timestamp_column=p_timestamp,
    object_types=p_object_types,
    # obj_separator=" AND ",
    additional_event_attributes=p_additional_event_attributes,
    additional_object_attributes=p_additional_object_attributes
)

save_file(ocel, "ocel")

number_of_events = len(ocel.events)

number_of_objects = len(ocel.objects)

number_of_activities = ocel.events['ocel:activity'].nunique()

object_columns = [col for col in ocel.events.columns if 'ocel:' in col and col !=
                  'ocel:activity' and col != 'ocel:timestamp']
unique_object_values = pd.concat(
    [ocel.events[col].dropna() for col in object_columns]).unique()
number_of_object_types = len(unique_object_values)

if 'related_objects' in ocel.events.columns:
    events_objects_relationships = ocel.events['related_objects'].apply(
        len).sum()
else:
    events_objects_relationships = 0

activities_occurrences = Counter(ocel.events['ocel:activity'])

statistics_ocel = (
    f"Object-Centric Event Log statistics: "
    f"Number of events: {number_of_events}, "
    f"number of objects: {number_of_objects}, "
    f"number of activities: {number_of_activities}, "
    f"number of object types: {number_of_object_types}, "
    f"events-objects relationships: {events_objects_relationships}\n"
    f"Activities occurrences: {activities_occurrences}"
)

print(statistics_ocel)

File spostati in: Datasets\pancacke100txs\2025_033
Object-Centric Event Log statistics: Number of events: 490, number of objects: 96, number of activities: 3, number of object types: 490, events-objects relationships: 0
Activities occurrences: Counter({'approve': 226, 'sendFrom': 168, 'transfer': 96})


In [42]:
combination = ocel.relations[[
    'ocel:type', 'ocel:activity', 'ocel:qualifier']].drop_duplicates()
display(combination)
qualifier_map = {}

for index, row in combination.iterrows():
    print(
        f"Riga corrente: ocel:type = {row['ocel:type']}, ocel:activity = {row['ocel:activity']}")
    qualifier = input("Inserisci il qualificatore per questa combinazione: ")
    qualifier_map[(row['ocel:type'], row['ocel:activity'])] = qualifier

ocel.relations['ocel:qualifier'] = ocel.relations.apply(
    lambda row: qualifier_map.get((row['ocel:type'], row['ocel:activity']), row['ocel:qualifier']), axis=1
)

,ocel:type,ocel:activity,ocel:qualifier
0,contractAddress,approve,None
1,sender,approve,None
2,input_inputName,approve,None
6,contractAddress,transfer,None
7,sender,transfer,None
8,input_inputName,transfer,None
36,contractAddress,sendFrom,None
37,sender,sendFrom,None
38,input_inputName,sendFrom,None
986,storageState_variableName,sendFrom,None


Riga corrente: ocel:type = contractAddress, ocel:activity = approve
Riga corrente: ocel:type = sender, ocel:activity = approve
Riga corrente: ocel:type = input_inputName, ocel:activity = approve
Riga corrente: ocel:type = contractAddress, ocel:activity = transfer
Riga corrente: ocel:type = sender, ocel:activity = transfer
Riga corrente: ocel:type = input_inputName, ocel:activity = transfer
Riga corrente: ocel:type = contractAddress, ocel:activity = sendFrom
Riga corrente: ocel:type = sender, ocel:activity = sendFrom
Riga corrente: ocel:type = input_inputName, ocel:activity = sendFrom
Riga corrente: ocel:type = storageState_variableName, ocel:activity = sendFrom


In [43]:
qualifier_map

{('contractAddress', 'approve'): 'contract_approve',
 ('sender', 'approve'): 'sender_approve',
 ('input_inputName', 'approve'): 'input_approve',
 ('contractAddress', 'transfer'): '',
 ('sender', 'transfer'): 'sender_transfer',
 ('input_inputName', 'transfer'): 'input_transfer',
 ('contractAddress', 'sendFrom'): 'contract_send',
 ('sender', 'sendFrom'): 'sender_send',
 ('input_inputName', 'sendFrom'): 'input_send',
 ('storageState_variableName', 'sendFrom'): 'storage_send'}

In [44]:
save_file(ocel, "ocel_qualifier")

File spostati in: Datasets\pancacke100txs\2025_033


## Se in esecuzione su google colab


In [1]:
%pip install pyecore > /dev/null
%pip install --upgrade pm4py > /dev/null
#%pip install web3 > /dev/null
from google.colab import drive, auth
import pandas as pd
import os
import datetime
import json
import pm4py
import shutil

/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba/cuda/cudadrv/driver.py", line 295, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))


In [2]:
drive.mount("/content/drive")
folder_path = "/content/drive/My Drive/Personal/University/Master\"s/Thesis Mengascini Michele/Raw data"

Mounted at /content/drive


In [3]:
file_list = [file for file in os.listdir(
    folder_path) if file.endswith(".json")]
print("File JSON disponibili:")
for i, file_name in enumerate(file_list):
    print(f"{i}: {file_name}")

if file_list:
    selected_index = int(
        input("Inserisci il numero corrispondente al file che vuoi scegliere: "))
    selected_file = file_list[selected_index]

    print(f"Hai selezionato: {selected_file}")
    file_path = os.path.join(folder_path, selected_file)
    print(f"Percorso completo del file: {file_path}")
else:
    print("Non ci sono file .json nella cartella.")

File JSON disponibili:
0: Oval3.json
1: FTM Fantom Miner.json
2: more_complex_activity_pancake.json
3: pancacke100txs.json
Inserisci il numero corrispondente al file che vuoi scegliere: 3
Hai selezionato: pancacke100txs.json
Percorso completo del file: /content/drive/My Drive/Personal/University/Master's/Thesis Mengascini Michele/Raw data/pancacke100txs.json


## Normalizzazione dataframe e salvataggio


In [4]:
with open(file_path, "r") as file:
    data = json.load(file)

meta_fields = ["txHash", "blockNumber", "contractAddress",
               "sender", "gasUsed", "activity", "timestamp"]


def normalize_data(data):
    # Normalizza i dati senza aggregazione
    inputs_df = pd.json_normalize(data, record_path="inputs", meta=meta_fields, record_prefix="input_")
    inputs_df["input__id"] = "input_" + inputs_df["txHash"] + "_" + (inputs_df.groupby("txHash").cumcount() + 1).astype(str)
    
    events_df = pd.json_normalize(data, record_path="events", meta=meta_fields, record_prefix="event_")
    events_df["event__id"] = "event_" + events_df["txHash"] + "_" + (events_df.groupby("txHash").cumcount() + 1).astype(str)
    
    storage_state_df = pd.json_normalize(data, record_path="storageState", meta=meta_fields, record_prefix="storageState_")
    storage_state_df["storageState__id"] = "storageState_" + storage_state_df["txHash"] + "_" + (storage_state_df.groupby("txHash").cumcount() + 1).astype(str)
    
    internal_txs_df = pd.json_normalize(data, record_path="internalTxs", meta=meta_fields, record_prefix="internalTx_")
    internal_txs_df["internalTx__id"] = "internalTx_" + internal_txs_df["txHash"] + "_" + (internal_txs_df.groupby("txHash").cumcount() + 1).astype(str)
    
    # Combina i DataFrame senza aggregazione
    combined_df = pd.concat(
        [inputs_df, events_df, storage_state_df, internal_txs_df], ignore_index=True)

    column_order = [
        "txHash", "blockNumber", "contractAddress", "sender", "gasUsed", "activity", "timestamp",
        
        "input_inputId", "input_inputName", "input_type", "input_inputValue", "input__id",
        
        "storageState_variableId", "storageState_variableName", "storageState_type",
        "storageState_variableValue", "storageState_variableRawValue", "storageState__id",
        
        "internalTx_callId", "internalTx_callType", "internalTx_to", 
        "internalTx_inputsCall", "internalTx__id",
        
        "event_eventId", "event_eventName", "event_eventValues.0", "event_eventValues.1",
        "event_eventValues.2", "event_eventValues.__length__", "event_eventValues.owner",
        "event_eventValues.spender", "event_eventValues.value", "event_eventValues.from",
        "event_eventValues.to", "event_eventValues.3", "event_eventValues._dstChainId",
        "event_eventValues._from", "event_eventValues._toAddress", "event_eventValues._amount",
        "event__id"
    ]
    
    existing_columns = [col for col in column_order if col in combined_df.columns]
    extra_columns = list(set(column_order) - set(existing_columns))
    
    if extra_columns:
        print(f"Warning: The following columns are not present in the data: {extra_columns}")

    return combined_df[existing_columns]


def remove_columns_containing_string(df, substring):
    filtered_columns = [col for col in df.columns if substring not in col]
    return df[filtered_columns]

df = pd.DataFrame(data)

# Esempio di utilizzo della funzione
df_normalized = normalize_data(data)
df_normalized = remove_columns_containing_string(df_normalized, "Id")
df_normalized["timestamp"] = pd.to_datetime(df_normalized["timestamp"])
df_normalized["blockNumber"] = df_normalized["blockNumber"].astype(int)
df_normalized["gasUsed"] = df_normalized["gasUsed"].astype(int)
df_normalized["activity"] = df_normalized["activity"].astype("category")
df_normalized["input_inputName"] = df_normalized["input_inputName"].astype("category")
df_normalized["input_type"] = df_normalized["input_type"].astype("category")
df_normalized["storageState_variableName"] = df_normalized["storageState_variableName"].astype("category")
df_normalized["storageState_type"] = df_normalized["storageState_type"].astype("category")
df_normalized["internalTx_callType"] = df_normalized["internalTx_callType"].astype("category")
df_normalized["event_eventName"] = df_normalized["event_eventName"].astype("category")

#print(df_normalized.columns)
#display(df_normalized)

In [5]:
df.memory_usage(deep=True).sum()

np.int64(92982)

## Conversione to ocel con PM4PY


[pm4py ocel ](https://processintelligence.solutions/static/api/2.7.11/api.html#object-centric-process-mining-pm4py-ocel)


### Mappature e OCEL


#### Mappatura 

In [7]:
tx_hash_value = "0x7cb0fcc73abd51cfecdd4bfb79eef7f3987c7b4bd645cb5c772f180877206ab4"

filtered_df = df.query("txHash == @tx_hash_value")
print(filtered_df.to_string())

print("\n")
filtered_df = df_normalized.query("txHash == @tx_hash_value")
print(filtered_df.to_string())

                                                               txHash blockNumber                             contractAddress                                      sender gasUsed activity                 timestamp                                                                                                                                                                                                                                                                                                                                                            inputs storageState internalTxs                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
import datetime
# Test
def save_file(ocel: pm4py.OCEL, file_name: str):
    year_day = datetime.datetime.utcnow().strftime("%Y_%j")
    hour_minute = datetime.datetime.utcnow().strftime("%H_%M")
    hour_minute = 0
    complete_file_name = f"{file_name}_{year_day}_{hour_minute}"
    jsonocel_file_path = os.path.join(
        folder_path, f"{os.path.splitext(selected_file)[0]}_{complete_file_name}")
    pm4py.write.write_ocel2_json(ocel, jsonocel_file_path + ".jsonocel")
    shutil.copy(jsonocel_file_path + ".jsonocel", jsonocel_file_path + ".json")
    subfolder_path = os.path.join(
        folder_path, os.path.splitext(selected_file)[0], year_day)
    os.makedirs(subfolder_path, exist_ok=True)
    if os.path.exists(jsonocel_file_path + ".jsonocel"):
        shutil.move(jsonocel_file_path + ".jsonocel", os.path.join(subfolder_path,
                    os.path.basename(jsonocel_file_path + ".jsonocel")))
    else:
        print(
            f"File {jsonocel_file_path + '.jsonocel'} non trovato per lo spostamento.")
    if os.path.exists(jsonocel_file_path + ".json"):
        shutil.move(jsonocel_file_path + ".json", os.path.join(subfolder_path,
                    os.path.basename(jsonocel_file_path + ".json")))
    else:
        print(
            f"File {jsonocel_file_path + '.json'} non trovato per lo spostamento.")
    print(f"File spostati in: {subfolder_path}")


p_log = df_normalized

p_activity = "activity"

p_timestamp = "timestamp"

p_object_types = [
    "contractAddress", "sender", "activity", "input__id", "storageState__id", "internalTx__id", "event__id"
]

p_additional_event_attributes = [
    "txHash", "blockNumber", "gasUsed", "input_inputName", "input_type", "input_inputValue",
    "event_eventName", "event_eventValues.0",
    "event_eventValues.1", "event_eventValues.2",
    "event_eventValues.__length__", "event_eventValues.owner",
    "event_eventValues.spender", "event_eventValues.value",
    "event_eventValues.from", "event_eventValues.to", "event_eventValues.3",
    "event_eventValues._from", "event_eventValues._toAddress",
    "event_eventValues._amount", "storageState_variableName",
    "storageState_type", "storageState_variableValue",
    "storageState_variableRawValue", "internalTx_callType", "internalTx_to",
    "internalTx_inputsCall"
]

p_additional_object_attributes = {
    "txHash": ["txHash", "blockNumber", "contractAddress", "sender", "gasUsed", "activity", "timestamp"],
    "blockNumber": ["blockNumber"],
    "contractAddress": ["contractAddress"],
    "sender": ["sender"],
    "activity": ["activity"],
    "input__id": ["input__id", "input_inputName", "input_type", "input_inputValue"],
    "storageState__id": ["storageState__id", "storageState_variableName", "storageState_type", "storageState_variableValue", "storageState_variableRawValue"],
    "internalTx__id": ["internalTx__id", "internalTx_callType", "internalTx_to", "internalTx_inputsCall"],
    "event__id": ["event__id"],
    "activity": ["activity"]
}


# Converti il DataFrame in un OCEL
ocel = pm4py.convert.convert_log_to_ocel(
    log=p_log,
    activity_column=p_activity,
    timestamp_column=p_timestamp,
    object_types=p_object_types,
    # obj_separator=" AND ",
    additional_event_attributes=p_additional_event_attributes,
    additional_object_attributes=p_additional_object_attributes
)


ocel.relations["ocel:qualifier"] = ocel.relations["ocel:type"]
# ocel_normalized.events["ocel:eid"] = ocel_normalized.events["txHash"]

print("\n\nOcel with duplicates:\n")
print(ocel)
save_file(ocel, "ocel_with_dupliacates")

# ocel_merge = pm4py.ocel.ocel_merge_duplicates(ocel.__copy__())
# # ocel_cluster = pm4py.ocel.cluster_equivalent_ocel(ocel_normalized.__copy__(), "contractAddress")

# print("\n\n")
# print("Ocel merged duplicates:\n")
# print(ocel_merge)
# save_file(ocel_merge, "ocel_merge_duplicates")
# print(ocel_normalized.relations.columns)
# print("\n\n")
# print(ocel_cluster)
# pm4py.write.write_ocel2_json(ocel_cluster, "ocel_cluster_duplicates")



Ocel with duplicates:

Object-Centric Event Log (number of events: 490, number of objects: 577, number of activities: 3, number of object types: 7, events-objects relationships: 1960)
Activities occurrences: Counter({'approve': 226, 'sendFrom': 168, 'transfer': 96})
Object types occurrences (number of objects): Counter({'input__id': 228, 'event__id': 150, 'internalTx__id': 91, 'sender': 83, 'storageState__id': 21, 'activity': 3, 'contractAddress': 1})
Unique activities per object type: Counter({'activity': 3, 'contractAddress': 3, 'event__id': 3, 'input__id': 3, 'sender': 3, 'internalTx__id': 1, 'storageState__id': 1})
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.
File spostati in: Datasets\pancacke100txs\2025_029


In [12]:
ocel.relations

,ocel:eid,ocel:activity,ocel:timestamp,txHash,blockNumber,gasUsed,input_inputName,input_type,input_inputValue,ocel:oid,...,event_eventValues._toAddress,event_eventValues._amount,storageState_variableName,storageState_type,storageState_variableValue,storageState_variableRawValue,internalTx_callType,internalTx_to,internalTx_inputsCall,ocel:qualifier
0,0,approve,2024-05-13 02:08:11+00:00,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,46225,spender,address,152649eA73beAb28c5b49B26eb48f7EAD6d4c898,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contractAddress
1,0,approve,2024-05-13 02:08:11+00:00,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,46225,spender,address,152649eA73beAb28c5b49B26eb48f7EAD6d4c898,0xf878d31f62a1e1f393554406659be60287e567b5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sender
2,0,approve,2024-05-13 02:08:11+00:00,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,46225,spender,address,152649eA73beAb28c5b49B26eb48f7EAD6d4c898,approve,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,activity
3,0,approve,2024-05-13 02:08:11+00:00,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,46225,spender,address,152649eA73beAb28c5b49B26eb48f7EAD6d4c898,input_0x1745af675d56831b92feafdc15b7270497b65c...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,input__id
4,1,approve,2024-05-13 02:08:11+00:00,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,46225,amount,uint256,2,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,contractAddress
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,488,sendFrom,2024-05-14 05:17:23+00:00,0xd5bfdf7973a36dbbd59ab0e2a5148bc6a3bb62ebd16c...,19866110,230165,NaN,NaN,NaN,internalTx_0xd5bfdf7973a36dbbd59ab0e2a5148bc6a...,...,NaN,NaN,NaN,NaN,NaN,NaN,DELEGATECALL,00000000000000000000000013dff8847ea170ebb8439c...,[c1723a1d0000000000000000000000000000000000000...,internalTx__id
1956,489,sendFrom,2024-05-14 05:17:23+00:00,0xd5bfdf7973a36dbbd59ab0e2a5148bc6a3bb62ebd16c...,19866110,230165,NaN,NaN,NaN,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,...,NaN,NaN,NaN,NaN,NaN,NaN,STATICCALL,0000000000000000000000003773e1e9deb273fcdf9f80...,[5cbbbd750000000000000000000000000000000000000...,contractAddress
1957,489,sendFrom,2024-05-14 05:17:23+00:00,0xd5bfdf7973a36dbbd59ab0e2a5148bc6a3bb62ebd16c...,19866110,230165,NaN,NaN,NaN,0xfdcc8aff007890cd3cf0be646be50898d3914835,...,NaN,NaN,NaN,NaN,NaN,NaN,STATICCALL,0000000000000000000000003773e1e9deb273fcdf9f80...,[5cbbbd750000000000000000000000000000000000000...,sender
1958,489,sendFrom,2024-05-14 05:17:23+00:00,0xd5bfdf7973a36dbbd59ab0e2a5148bc6a3bb62ebd16c...,19866110,230165,NaN,NaN,NaN,sendFrom,...,NaN,NaN,NaN,NaN,NaN,NaN,STATICCALL,0000000000000000000000003773e1e9deb273fcdf9f80...,[5cbbbd750000000000000000000000000000000000000...,activity


In [11]:
# display(ocel_normalized.events.head())
# display(ocel_normalized.relations.head())
display(ocel_normalized.objects.head())

,ocel:oid,ocel:type,contractAddress,sender,activity,input__id,input_inputName,input_type,input_inputValue,event__id,storageState__id,storageState_variableName,storageState_type,storageState_variableValue,storageState_variableRawValue,internalTx__id,internalTx_callType,internalTx_to,internalTx_inputsCall
0,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,contractAddress,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xf878d31f62a1e1f393554406659be60287e567b5,sender,NaN,0xf878d31f62a1e1f393554406659be60287e567b5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,approve,activity,NaN,NaN,approve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,input_0x1745af675d56831b92feafdc15b7270497b65c...,input__id,NaN,NaN,NaN,input_0x1745af675d56831b92feafdc15b7270497b65c...,spender,address,152649eA73beAb28c5b49B26eb48f7EAD6d4c898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,input_0x1745af675d56831b92feafdc15b7270497b65c...,input__id,NaN,NaN,NaN,input_0x1745af675d56831b92feafdc15b7270497b65c...,amount,uint256,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Mappatura proposta

In [ ]:
# Mappatura proposta, da adattare per utilizzare il df normalizzato

p_log = df_normalized

p_activity = "activity"

p_timestamp = "timestamp"

p_object_types = [
    "contractAddress"
]

p_additional_event_attributes = [
    "input_inputName", "input_type", "input_inputValue",
    "txHash", "blockNumber", "contractAddress", "sender", "gasUsed",
    "activity", "timestamp", "event_eventName",
    "event_eventValues.0", "event_eventValues.1", "event_eventValues.2",
    "event_eventValues.__length__", "event_eventValues.owner",
    "event_eventValues.spender", "event_eventValues.value",
    "event_eventValues.from", "event_eventValues.to", "event_eventValues.3", "event_eventValues._from",
    "event_eventValues._toAddress", "event_eventValues._amount",
    "storageState_variableName",
    "storageState_type", "storageState_variableValue",
    "storageState_variableRawValue",
    "internalTx_callType", "internalTx_to", "internalTx_inputsCall"
]

p_additional_object_attributes = {
    "contractAddress": ["contractAddress"]
}

In [ ]:
p_log = df_normalized_and_aggregated
p_activity = "activity"
p_timestamp = "timestamp"

p_object_types = [
    "contractAddress",
    "txHash",
    "sender",
    "input_inputId",
    "storageState_variableName",
    "event_eventName",
    "internalTx_callType"
]

p_additional_event_attributes = [
    "gasUsed",
    "sender",
    "blockNumber",
    "contractAddress",
    "activity",
    "timestamp"
]

p_additional_object_attributes = {
    # "contractAddress": ["blockNumber", "gasUsed", "sender"],
    # "txHash": ["blockNumber", "sender", "gasUsed"],
    # "sender": ["blockNumber", "contractAddress", "gasUsed"],
    "input_inputId": ["input_inputName", "input_inputValue", "input_type"],
    # "storageState_variableId": ["storageState_variableValue", "storageState_variableRawValue"],
    # "storageState_variableName": ["storageState_variableValue", "storageState_variableRawValue"],
    # "event_eventName": ["event_eventValues.value"],
    # "internalTx_callId": ["internalTx_to", "internalTx_inputsCall", "internalTx_callType"],
    # "internalTx_callType": ["internalTx_to", "internalTx_inputsCall", "internalTx_callId"]
}

#### Generazione ocel

In [ ]:
# Converti il DataFrame in un OCEL
ocel = pm4py.convert.convert_log_to_ocel(
    log=p_log,
    activity_column=p_activity,
    timestamp_column=p_timestamp,
    object_types=p_object_types,
    # obj_separator=" AND ",
    additional_event_attributes=p_additional_event_attributes,
    additional_object_attributes=p_additional_object_attributes
)

ocel

Object-Centric Event Log (number of events: 100, number of objects: 100, number of activities: 3, number of object types: 1, events-objects relationships: 100)
Activities occurrences: Counter({'approve': 61, 'transfer': 32, 'sendFrom': 7})
Object types occurrences (number of objects): Counter({'txHash': 100})
Unique activities per object type: Counter({'txHash': 3})
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

#### Display columns


In [ ]:
display(ocel.events.dtypes)  # Tipi di dati delle colonne degli eventi
display(ocel.events.head())  # Esempio dei primi eventi

,0
ocel:eid,string[python]
ocel:activity,string[python]
ocel:timestamp,object
inputs,object
storageState,object
internalTxs,object
events,object


,ocel:eid,ocel:activity,ocel:timestamp,inputs,storageState,internalTxs,events
0,0,approve,2024-05-13T02:08:11.000Z,[{'inputId': 'inputName_0_0x1745af675d56831b92...,[],[],[{'eventId': 'event_0_0x1745af675d56831b92feaf...
1,1,transfer,2024-05-13T03:00:47.000Z,[{'inputId': 'inputName_2_0x047dd1cd5b4fe326b8...,[],[],[{'eventId': 'event_1_0x047dd1cd5b4fe326b8f5e6...
2,2,approve,2024-05-13T03:29:59.000Z,[{'inputId': 'inputName_4_0xee2bb7bf70b70df3d1...,[],[],[{'eventId': 'event_2_0xee2bb7bf70b70df3d116b6...
3,3,approve,2024-05-13T03:38:35.000Z,[{'inputId': 'inputName_6_0x4a10361dcf3f6fe316...,[],[],[{'eventId': 'event_3_0x4a10361dcf3f6fe316601a...
4,4,approve,2024-05-13T03:42:59.000Z,[{'inputId': 'inputName_8_0x7cb0fcc73abd51cfec...,[],[],[{'eventId': 'event_4_0x7cb0fcc73abd51cfecdd4b...


In [13]:
display(ocel_normalized.objects.dtypes)  # Tipi di dati delle colonne degli oggetti
display(ocel_normalized.objects.head())  # Esempio dei primi eventi

ocel:oid                         string[python]
ocel:type                        string[python]
contractAddress                          object
sender                                   object
activity                                 object
input__id                                object
input_inputName                          object
input_type                               object
input_inputValue                         object
event__id                                object
storageState__id                         object
storageState_variableName                object
storageState_type                        object
storageState_variableValue               object
storageState_variableRawValue            object
internalTx__id                           object
internalTx_callType                      object
internalTx_to                            object
internalTx_inputsCall                    object
dtype: object

,ocel:oid,ocel:type,contractAddress,sender,activity,input__id,input_inputName,input_type,input_inputValue,event__id,storageState__id,storageState_variableName,storageState_type,storageState_variableValue,storageState_variableRawValue,internalTx__id,internalTx_callType,internalTx_to,internalTx_inputsCall
0,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,contractAddress,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0xf878d31f62a1e1f393554406659be60287e567b5,sender,NaN,0xf878d31f62a1e1f393554406659be60287e567b5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,approve,activity,NaN,NaN,approve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,input_0x1745af675d56831b92feafdc15b7270497b65c...,input__id,NaN,NaN,NaN,input_0x1745af675d56831b92feafdc15b7270497b65c...,spender,address,152649eA73beAb28c5b49B26eb48f7EAD6d4c898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,input_0x1745af675d56831b92feafdc15b7270497b65c...,input__id,NaN,NaN,NaN,input_0x1745af675d56831b92feafdc15b7270497b65c...,amount,uint256,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Salvataggio jsonocel e json


In [81]:
def save_file(ocel: pm4py.OCEL):
    timestamp = datetime.datetime.utcnow().strftime("%Y_%j_%H_%M")
    jsonocel_file_path = folder_path + \
        f"/{os.path.splitext(selected_file)[0]}_{timestamp}"

    pm4py.write.write_ocel2_json(ocel, jsonocel_file_path)

    shutil.copy(jsonocel_file_path+".jsonocel", jsonocel_file_path+".json")

    subfolder_path = os.path.join(
        folder_path, os.path.splitext(selected_file)[0], timestamp)

    os.makedirs(subfolder_path, exist_ok=True)

    if os.path.exists(jsonocel_file_path+".jsonocel"):
        shutil.move(jsonocel_file_path+".jsonocel", os.path.join(subfolder_path,
                    os.path.basename(jsonocel_file_path+".jsonocel")))
    else:
        print(
            f"File {jsonocel_file_path+".jsonocel"} non trovato per lo spostamento.")

    if os.path.exists(jsonocel_file_path+".json"):
        shutil.move(jsonocel_file_path+".json", os.path.join(subfolder_path,
                    os.path.basename(jsonocel_file_path+".json")))
    else:
        print(f"File {jsonocel_file_path+".json"} non trovato per lo spostamento.")

    print(f"File spostati in: {subfolder_path}")

## Test


In [ ]:
# Gruppo le righe per tipo di attività
grouped = df.groupby("activity")

# Controllo quali colonne sono non-null per ciascun tipo di attività
non_null_summary = grouped.apply(lambda x: x.notnull().any()).transpose()

# Trovo le colonne specifiche per un solo tipo di attività
unique_columns = non_null_summary[non_null_summary.sum(axis=1) == 1]

# Mappo le colonne al rispettivo tipo di attività
activity_to_columns = {
    activity: unique_columns[unique_columns[activity]].index.tolist()
    for activity in unique_columns.columns
    if not unique_columns[unique_columns[activity]].empty
}

print("Colonne specifiche per ogni attività:")
for activity, columns in activity_to_columns.items():
    print(f"Attività "{activity}": {columns}")

Colonne specifiche per ogni attività:
Attività 'approve': ['event_eventValues.owner', 'event_eventValues.spender']
Attività 'sendFrom': ['storageState_variableId', 'storageState_variableName', 'storageState_type', 'storageState_variableValue', 'storageState_variableRawValue', 'internalTx_callId', 'internalTx_callType', 'internalTx_to', 'internalTx_inputsCall', 'event_eventValues.3', 'event_eventValues._dstChainId', 'event_eventValues._from', 'event_eventValues._toAddress', 'event_eventValues._amount']


<ipython-input-122-ee8565181d9d>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  non_null_summary = grouped.apply(lambda x: x.notnull().any()).transpose()


### Importazione automatica di transaction da blockchain


In [ ]:
from web3 import Web3

# Replace with your Ethereum node URL (Infura, Alchemy, or your local node)
node_url = "https://mainnet.infura.io/v3/YOUR_INFURA_PROJECT_ID"

# Connect to the Ethereum node
web3 = Web3(Web3.HTTPProvider(node_url))

if web3.isConnected():
    print("Connected to Ethereum node")
else:
    raise Exception("Failed to connect to Ethereum node")

# Get the latest block number
latest_block = web3.eth.block_number
print(f"Latest Block: {latest_block}")

# Fetch the latest 100 transactions
num_transactions_to_fetch = 100
transactions = []

# Iterate over the blocks until we collect enough transactions
while len(transactions) < num_transactions_to_fetch:
    block = web3.eth.get_block(latest_block, full_transactions=True)

    # Append transactions from the block
    transactions.extend(block.transactions)

    # Move to the previous block
    latest_block -= 1

    if latest_block < 0:  # Safety check to avoid invalid blocks
        break

# Limit to the desired number of transactions
latest_transactions = transactions[:num_transactions_to_fetch]

# Display transaction details
for tx in latest_transactions:
    print(f"Tx Hash: {tx.hash.hex()}")
    print(f"From: {tx["from"]}")
    print(f"To: {tx["to"]}")
    print(f"Value: {web3.from_wei(tx["value"], "ether")} ETH")
    print(f"Gas: {tx["gas"]}")
    print(f"Gas Price: {web3.from_wei(tx["gasPrice"], "gwei")} GWEI")
    print("----")

Object-Centric Event Log (number of events: 490, number of objects: 84, number of activities: 3, number of object types: 2, events-objects relationships: 980)
Activities occurrences: {'approve': 226, 'sendFrom': 168, 'transfer': 96}
Object types occurrences (number of objects): {'sender': 83, 'contractAddress': 1}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

In [ ]:
df_normalized

,txHash,blockNumber,contractAddress,sender,gasUsed,activity,timestamp,input_inputId,input_inputName,input_type,...,event_eventValues.owner,event_eventValues.spender,event_eventValues.value,event_eventValues.from,event_eventValues.to,event_eventValues.3,event_eventValues._dstChainId,event_eventValues._from,event_eventValues._toAddress,event_eventValues._amount
0,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0xf878d31f62a1e1f393554406659be60287e567b5,46225,approve,2024-05-13T02:08:11.000Z,inputName_0_0x1745af675d56831b92feafdc15b72704...,spender,address,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x1745af675d56831b92feafdc15b7270497b65c6b55b0...,19858013,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0xf878d31f62a1e1f393554406659be60287e567b5,46225,approve,2024-05-13T02:08:11.000Z,inputName_1_0x1745af675d56831b92feafdc15b72704...,amount,uint256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x047dd1cd5b4fe326b8f5e6df778fcf4088b6202c9013...,19858275,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0xa08da07e046303b21d9efeca0661c2fd959210e3,46782,transfer,2024-05-13T03:00:47.000Z,inputName_2_0x047dd1cd5b4fe326b8f5e6df778fcf40...,to,address,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x047dd1cd5b4fe326b8f5e6df778fcf4088b6202c9013...,19858275,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0xa08da07e046303b21d9efeca0661c2fd959210e3,46782,transfer,2024-05-13T03:00:47.000Z,inputName_3_0x047dd1cd5b4fe326b8f5e6df778fcf40...,amount,uint256,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0xee2bb7bf70b70df3d116b6e3669dbd3032acfc27a1ec...,19858421,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0x53e883570510db3fd97f664d849c99285d7103cb,46321,approve,2024-05-13T03:29:59.000Z,inputName_4_0xee2bb7bf70b70df3d116b6e3669dbd30...,spender,address,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,0xc6c286b0431e34f9075e77f00fcdbb89df77c058ba8f...,19868090,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0x8dbd3b127ddca7eabddf913666a90a7c706af2a1,46597,approve,2024-05-14T11:57:11.000Z,NaN,NaN,NaN,...,0x8dBd3b127DDcA7EAbddF913666A90a7c706Af2A1,0x881D40237659C251811CEC9c364ef91dC08D300C,1157920892373161954235709850086879078532699846...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,0x6d3cf950712e5e6d421a32ea1051252c4c9cdc63e20a...,19868214,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0x920d725e2f8d8b0502fffbf594f8b05326c3992c,46321,approve,2024-05-14T12:22:23.000Z,NaN,NaN,NaN,...,0x920D725E2f8d8B0502FfFbF594F8b05326c3992c,0x1111111254EEB25477B68fb85Ed929f73A960582,37284247135849110000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
487,0x6d3cf950712e5e6d421a32ea1051252c4c9cdc63e20a...,19868214,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0x920d725e2f8d8b0502fffbf594f8b05326c3992c,46321,approve,2024-05-14T12:22:23.000Z,NaN,NaN,NaN,...,0x920D725E2f8d8B0502FfFbF594F8b05326c3992c,0x1111111254EEB25477B68fb85Ed929f73A960582,1775440339802340000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
488,0x6d3cf950712e5e6d421a32ea1051252c4c9cdc63e20a...,19868214,0x152649ea73beab28c5b49b26eb48f7ead6d4c898,0x920d725e2f8d8b0502fffbf594f8b05326c3992c,46321,approve,2024-05-14T12:22:23.000Z,NaN,NaN,NaN,...,NaN,NaN,35508806796046770000,0x920D725E2f8d8B0502FfFbF594F8b05326c3992c,0xE37e799D5077682FA0a244D46E5649F71457BD09,NaN,NaN,NaN,NaN,NaN


In [17]:
%pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [49]:
df_normalized["blockNumber"].nunique()

99

In [43]:
df_normalized["input_inputName"].unique()

['spender', 'amount', 'to', '_from', '_dstChainId', '_toAddress', '_amount', '_minAmount', '_callParams,refundAddress,zroPaymentAddress,a..., NaN]
Categories (9, object): ['_amount', '_callParams,refundAddress,zroPaymentAddress,a..., '_dstChainId', '_from', ..., '_toAddress', 'amount', 'spender', 'to']